# علم داده در فضای ابری: روش "Azure ML SDK"

## مقدمه

در این دفترچه یادداشت، یاد خواهیم گرفت که چگونه از Azure ML SDK برای آموزش، استقرار و استفاده از یک مدل از طریق Azure ML استفاده کنیم.

پیش‌نیازها:
1. شما یک فضای کاری Azure ML ایجاد کرده‌اید.
2. شما مجموعه داده [Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) را در Azure ML بارگذاری کرده‌اید.
3. شما این دفترچه یادداشت را در Azure ML Studio آپلود کرده‌اید.

مراحل بعدی عبارتند از:

1. ایجاد یک آزمایش در یک فضای کاری موجود.
2. ایجاد یک خوشه محاسباتی.
3. بارگذاری مجموعه داده.
4. پیکربندی AutoML با استفاده از AutoMLConfig.
5. اجرای آزمایش AutoML.
6. بررسی نتایج و دریافت بهترین مدل.
7. ثبت بهترین مدل.
8. استقرار بهترین مدل.
9. استفاده از نقطه پایانی.

## واردات خاص Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## مقداردهی اولیه فضای کاری  
یک شی فضای کاری را از پیکربندی ذخیره‌شده مقداردهی اولیه کنید. اطمینان حاصل کنید که فایل پیکربندی در مسیر .\config.json موجود باشد.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## ایجاد یک آزمایش Azure ML

بیایید یک آزمایش به نام 'aml-experiment' در فضای کاری که تازه مقداردهی اولیه کردیم، ایجاد کنیم.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## ایجاد یک کلاستر محاسباتی  
شما نیاز دارید یک [هدف محاسباتی](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) برای اجرای AutoML خود ایجاد کنید.  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## داده‌ها  
اطمینان حاصل کنید که مجموعه داده را به Azure ML آپلود کرده‌اید و کلید همان نام مجموعه داده باشد.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## تنظیمات AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## اجرای AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## استقرار بهترین مدل

کد زیر را اجرا کنید تا بهترین مدل مستقر شود. می‌توانید وضعیت استقرار را در پورتال Azure ML مشاهده کنید. این مرحله ممکن است چند دقیقه طول بکشد.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## استفاده از نقطه پایانی
می‌توانید ورودی‌ها را به نمونه ورودی زیر اضافه کنید.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
